In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

# Attention: motivation

Let us consider a task familiar to all of us who have taken standardized exams: Question Answering.

Input consists of two pieces of text
- a paragraph (called the *Context*)
- a Question whose answer can be found in the paragraph

Output is a piece of text that answers the question.

$
\x = \;
\begin{Bmatrix}\\
\text{Context:} & \text{The FRE Dept offers many Spring classes.  The students are great. ...} \\
& \vdots \\
& \text{Professor Perry taught them Machine Learning. The students ...}, \\
& \vdots \\
& \text{Professor Blecherman led a class in ...} \\
& \vdots \\
\text{Question:} & \text{What did Professor Perry do ?} \\
\end{Bmatrix}
$
<br><br><br>
$
\y = \;
\begin{array} \\
\text{Answer:} & \text{He taught them Machine Learning}
\end{array}
$

This is an example of a *Sequence to Sequence* task
- briefly encountered when we defined RNN's

Language translation is another common Sequence to Sequence task.

Let us hypothesize how a model might learn to solve this task
- it is only an hypothesis: we don't really know

The model might have generalized
- from seeing many training examples of disparate questions and their answers
- that there is a parameterized *template* for both the Question and the Answer

Question Template

> What did Professor `<PROPER NOUN>` teach in the Spring ?

Answer Template:
```
<PRONOUN> <VERB> <INIDRECT OBJECT> <OBJECT>
```

where `<PROPER NOUN>, <PRONOUN>, <VERB>`, etc. are *pattern place-holders* parameters.

By using a parameterized template, the model captures
- commonality
- in many different types of questions

In order to produce the answer the model needs to
- Generate the tokens of the Answer Template in order
- Substituting in concrete values for the place-holders
    - by performing a Lookup in the Context in order to obtain these values

We will examine how the Lookup might be performed
- first: by using mechanisms that we have already studied
- subsequently: via a new mechanism called *Attention*


# Using an RNN without Attention

## Encoder-Decoder architecture: review

For the Question Answering task
- both the Input and Output are sequences
- thus, the task is a Sequence to Sequence task
    - just like: Language Translation

We learned that Recurrent architectures are best-suited for processing sequences.

These architectures
- operate in a "loop"
    - processing one Input or Output token at a time
- utilize **memory** (latent state)
    - necessary because Input/Output sequence lengths are unbounded
    - after processing the token at position $\tt$
        - the latent state is finite representation of the prefix of the sequence of length $\tt$

Loop architecture

- Uses a "latent state" that is updated with each element of the sequence, then predict the output

$$
\begin{array}[lll] \\
\pr{\h_\tp | \x_\tp, \h_{(\tt-1)} } & \text{latent variable } \h_\tp \text{encodes } [ \x_{(1)} \dots \x_\tp ]\\
\pr{\hat\y_\tp | \h_\tp }              & \text{prediction contingent on latent variable} \\
\end{array}
$$

    
<br>
<div>
    <center><strong>Loop with latent state</strong></center>
    <img src="images/RNN_arch_loop.png" width=70%>
</div>


A common architecture for Sequence to Sequence tasks is the Encoder-Decoder:
- The Encoder is an RNN
    - Acts on input sequence $[\x_{(1)} \dots \x_{(\bar{T})}]$
    - Producing a sequence of latent states $[ \bar{\h}_{(1)}, \dots, \bar{\h}_{(\bar{T})} ]$
        - latent state $\bar{\h}_\tt$ is a summary of $[\x_{(1)} \dots \x_\tp ]$


The Decoder
- Acts on the *final* Encoder latent state $\bar{\h}_{(\bar{T})}$
    - which summarizes the entire input sequence $\x$
    - Producing a sequence of latent states $[ \h_{(1)}, \dots, \h_{(T)} ]$
        - latent state $\h_\tp$ is response for generating output token $\hat{\y}_\tp$
    - Thus outputting a sequence  $[ \hat{\y}_{(1)}, \dots, \hat{\y}_{(T)} ]$
- Often feeding step $\tt$ output $\hat{\y}_\tp$ as Encoder input at step $(\tt+1)$



<table>
    <tr>
        <th><center>RNN Encoder/Decoder</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_API_Encoder_Decoder.png"</td>
    </tr>
</table>


## Decoder output $\hat\y_\tp$

The simplest RNN (corresponding to our diagrams) use the latent state $\h_\tp$ as the output $\hat\y_\tp$
$$
\hat\y_\tp = \h_\tp
$$

It is easy to add another NN to transform $\h_\tp$ into a $\hat\y_\tp$ that is different.

- We can add a NN to the Decoder RNN that implements a function $D$ that transforms the latent state into an output.

$$\hat\y_\tp = D(\h_\tp)$$

For clarity: we will omit this additional NN from our diagrams until it becomes necessary

Here is what the additional NN looks like:
<br>
<br>



<table>
    <tr>
        <th><center>Decoder output transformation: No attention</center></th>
    </tr>
    <tr>
        <td><img src="images/Decoder_no_attention.png" width=70%></td>
    </tr>
</table>

## How does the Decoder perform Lookup (without Attention) ?

Suppose the Decoder has already output 
$$\hat\y_{([1:3])} = \text{He taught them}$$

It must subsequently output
$$
\hat\y_{([4:5])} = \text{Machine Learning}$$

In order to do this
- it must Lookup "Machine Learning" in the Context, resulting in
$$\begin{array} \\
D( \h_{(4)}) & = & \text{Machine} \\
D( \h_{(5)}) & = & \text{Learning} \\
\end{array}
$$

But  $D$ is conditioned on the single input $\h_\tp$.

Thus, in order for $D( \h_{(4)} )$ to be equal to "Machine"
- this information must somehow be encoded in $\h_{(4)}$


How did it get there ?

All "knowledge" from the Context must be transfered from Encoder to Decoder
- through final Encoder state $\bar \h_{(\bar T)}$
- which in turn was encoded in all Encoder states $\bar \h_{({\bar \tt}')} \text{ for } \tt' \ge \bar p$
    - where $\bar p$ is the position withing sequence $\x$ of the word "Machine"

We can hypothesize that the final Encoder latent state $\bar\h_{\bar T}$
- encodes a Dictionary (key/value pairs)
- mapping Place-holder names to Concrete values
- the dictionary is built incrementally by prior latent states of the Encoder
<br>
<table>
    <tr>
        <th><center>Answering questions using Attention</center></th>
    </tr>
    <tr>
        <td><img src="images/Attention_example_1.png" width=80%></td>
    </tr>
</table>

This dictionary is passed to the Decoder via the single connection from Encoder to Decoder
- and must be carried forward by the Decoder
- through Decoder states $[ \h_{(1)}, \dots, \h_{(4)} ]$
- in order to make the dictionary available to subsequent latent states of the Decoder

We further hypothesize that the Decoder
- performs Lookups 
- by using the Decoder latent state $\h_\tp$
    - as a *query* that matches against the keys of the Dictionary
    - in order to obtain the Concrete value required to produced output token at position $\tt$
    $$\hat\y_\tp = D(\h_\tp)$$

<br>
<br>
<table>
    <tr>
        <th><center>Query performing a Lookup in the Dictionary</center></th>
    </tr>
    <tr>
        <td><img src="images/Attention_example_2.png" width=80%></td>
    </tr>
</table>

Here is a picture describing this hypothetical functioning.
<br>
<br>
<table>
    <tr>
        <th><center>RNN Encoder/Decoder without Attention<br>Bottleneck</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_API_Encoder_Decoder_1.png" width=80%</td>
    </tr>
    <tr>
        <td><img src="images/Encoder_Decoder_no_attention.png" width=70%</td>
</table>

Connecting the Encoder and Decoder through the "bottleneck" of $\bar \h_{(\bar T)}$ thus burdens the
- Encoder: passing knowledge forward to the bottleneck
- Decoder: passing knowledge from the bottleneck
    - to all subsequent positions (red box within each circle)

This results in an inefficient use of the model's latent state variable
- In addition to
    - the Encoder and Decoder allocating some of the model's latent state for "control"
    - guiding the loop that processes the Input, or generates the output positions in the template
- It must **also** allocate some of the model's latent state for "knowledge storage"
    - in order to Lookup the concrete value corresponding to a place-holder in the Output template


# Cross Attention

**Reference**

[Neural Machine Translation by Jointly Learning To Align and Translate](https://arxiv.org/pdf/1409.0473.pdf)
[paper that introduced Attention](https://arxiv.org/pdf/1409.0473.pdf)

<br><br>
The flaw in the Encoder-Decoder without Attention is 
- the input $\x$ is processed *only once*
- by the Encoder
- which has to summarize it in $\bar{\h}_{(\bar T)}$

We will introduce a mechanism called *Attention*
- that allows the input sequence to be *re-visited* at each time step of Output generation

This will result in a cleaner separation between control memory and input memory

Attention allows the Decoder
- to directly access all of the Encoder latent states $\bar\h_{(1)} \dots \h_{(\bar{T})}$
- at each time step of the Decoder

Thus, there is no need
- for an Encoder to create a full dictionary as the final Encoder latent state $\bar\h_{(\bar T)}$
- for the Decoder to keep the dictionary in all it's latent states $\h_{(1)} \dots \h_{(T)}$

Here is a picture of an Encoder/Decoder augmented with Attention
- we have add an additional box to the diagram for the NN that implements the function $D$
    - that maps $\h_\tp$ to $\y_\tp$
    


<br>
<br>
<table>
    <tr>
        <th><center>RNN Encoder/Decoder with Attention</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_API_Encoder_Decoder_Attention_1.png" width=80%</td>
    </tr>
    <tr>
        <td><img src="images/Encoder_Decoder_no_attention.png" width=70%</td>
</table>

This is one "flavor" of Attention known as *Cross Attention*
- one component (the Decoder) uses as input (*attends to*) the output of another component (the Encoder)

Decoder has *direct access* to **all** outputs (i.e., Latent sates) of the Encoder
- each Encoder output is proxy for a prefix of the input
    
The pink box is the sequent of Encoder outputs
$$
\bar\h_{(1:\bar T)}
$$

Notice that the final Encoder latent state $\bar\h_{(\bar T)}$ is **no longer**  connected to the Decoder.

What is going on inside the "box" implementing function $D$ that we added at each time step ?

The box's input at step $\tt$
- the Decoder latent state $\h_\tp$
- the collection of Encoder latent states $\bar\h_{(1)} \dots \h_{(\bar{T})}$
    - the red box in the above diagram

That is, it is computing a $\hat\y_\tp$ that is a function of both $\h_\tp$ and $\bar\h_{(1)} \dots \h_{(\bar{T})}$

$$
\hat\y_\tp = D( \h_\tp,  [ \bar\h_{(1)} \dots \h_{(\bar{T})} ])
$$

# Self-Attention: removing the Encoder loop

There is an alternative to the loop architecture for processing sequences
- the direct function approach

The alternative to the loop was to create a "direct function"
- Taking a **sequence** $\x_{(1 \dots \tt)}$ as input
- Outputting $\hat\y_\tp$

<br>
<div>
    <center><strong>Direct function</strong></center>
    <img src="images/RNN_arch_parallel.png" width=50%>
</div>




Note that output at position $\tt$ *no longer depend* on latent state from position $(\tt-1)$
- we have removed the loop !

Thus, the Encoder can output *all* elements of sequence $\bar \h$ *simultaneously*
- n.b. the Encoder output sequence will still be denoted
$$\bar\h_{(1:\bar T)}$$

even though the outputs are no longer *latent states*

- each output position is independent of previous output
- only dependent on input

We removed the "loop" architecture of the Encoder by using  the direct function approach
- the mechanism enabling each position of the Encoder output to *attend* to the entire sequence $x$ is called *Self-Attention*
    - Notice: no dependency arrow between circles in the Encoder
- Encoder output is a direct function of **all** positions in the input
    - all Encoder output positions can be computed *in parallel*

The blue box represents the *entire* input sequence
$$
\x_{(1:\bar T)}
$$
We no longer refer to the Encoder output as a Latent state
- no more loop !

<table>
    <tr>
        <th><center>RNN Encoder/Decoder with Cross Attention/Decoder Self Attention</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_API_Encoder_Decoder_Attention_Encoder_Self_Attention.png"
             width=80%</td>
    </tr>
   
</table>

# Masked Self Attention: removing the Decoder loop

Finally we remove the loop architecture for the Decoder as well using
a different "flavor" of Self-Attention

The grey box represents the *entire* output sequence
$$
\hat\y_{(1:T)}
$$



<table>
    <tr>
        <th><center>RNN Encoder/Decoder with Cross Attention and Self Attention (Encoder/Decoder)</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_API_Encoder_Decoder_Attention_All_Self_Attention.png"
             width=80%</td>
    </tr>
   
</table>

Now
- the output sequence $\hat\y$ is built iteratively (auto-regressively)
- units work in parallel
- each iteration outputs *all* positions
$$
\hat\y_{(1:T)}
$$
    - including ones whose full inputs have not been defined yet!
    - $\hat\y_\tp$ is not defined until iteration $\tt$

This is confusing !

The point is we don't output position $\tt$ to the user until iteration $\tt$

We certainly don't want $\hat\y_\tp$ to change
on iterations $\tt' \gt \tt$
- don't want future outputs $\hat\y_{(\tt')}$ for $\tt' \ge \tt$ to affect $\hat\y_\tp$ 
- $\hat\y_\tp$ depends *only* on $\hat\y_{(1:\tt-1)}$

We can ensure this by using **Masked Self Attention**
- position $\tt$ can only access positions $\tt' \lt \tt$
$$
\hat\y_{(1:\tt-1)}
$$

This means that outputs after iteration $\tt$ *can't effect* $\hat\y_\tp$

# Visualizing Attention

We can illustrate the behavior of Neural Networks that have been augmented with Attention through diagrams.
- at a particular output position $\tt$
- we can display the amount of "attention"
- that each position in the input receives

## Visualizing Self-Attention

Self Attention can be used to create a Context Sensitive Encoding of words
- The meaning of a word may change depending on the rest of the sentence

We can illustrate this with an example: how the meaning of the word "it" changes
- The thickness of the blue line indicates the attention weight that is given in processing the word "it".

<img src=https://1.bp.blogspot.com/-AVGK0ApREtk/WaiAuzddKVI/AAAAAAAAB_A/WPV5ropBU-cxrcMpqJBFHg73K9NX4vywwCLcBGAs/s1600/image2.png>

Much of the recent advances in NLP may be attributed to these improved, context sensitive embeddings.

We note that simple Word Embeddings
- also capture "meaning"
- but are *not* sensitive to context

## Visualizing Cross Attention

## The Entailment task

The *Entailment* task is a binary classification task based on two sentence
- first sentence (the *premise*)
- second sentence (the *hypothesis*)

Classify: 
Does the Hypothesis logically follow (is *entailed* by) the Premise ?

Here is an illustration of what part of the Premise is attended to
as we encounter the Hypothesis

<br>
<center><strong>Attention: Entailment</strong></center>
<table>
    <tr>
        <td><img src="images/Attention_visualization_Entailment.png"></td>
    </tr>
    <tr>
        <td><center>Does the Premise logically entail the Hypothesis.</center></td>
    </tr>
</table>

Attribution: https://arxiv.org/pdf/1509.06664.pdf#page=6"

## The Date Normalization task

- Source: Dates in free-form: "Saturday 09 May 2018"
- Target: Dates in normalized form: "2018-05-09"

[link](https://github.com/datalogue/keras-attention#example-visualizations)

## The  Image captioning task

- Source: Image
- Target: Caption: "A woman is throwing a **frisbee** in a park."
- Attending over *pixels* **not** sequence

<center><strong>Visual attention</strong></center>
<table>
    <tr>
        <td><img src="images/shat_-002-027.jpg"></td>
        <td><img src="images/shat_-002-028.jpg"></td>
    </tr>
    <tr>
        <td colspan=2><center>A woman is throwing a <strong>frisbee</strong> in a park.</center></td>
    </tr>
</table>
Attribution: https://arxiv.org/pdf/1502.03044.pdf


# Implementing Attention (Preview): Inside the NN for $D$

Recall the Decoder conditions output $\hat\y_\tp$ on
- Decoder state $\h_\tp$
- Encoder output sequence $\bar \h_{(1:\bar T)}$

Here is a high-level view of the inner workings of the NN for $D$:
    
<br>
<br>
<table>
    <tr>
        <th><center>Decoder output transformation with attention</center></th>
    </tr>
    <tr>
        <td><img src="images/Decoder_attention.png" width=60%></td>
    </tr>
</table>

Inside the box:
- the Decoder latent state $\h_\tp$ is used as a *query*
- which is matched against each of the Encoder outputs
- resulting in one Encoder output being chosen as $\mathbf{c}_\tp$

The chosen Encoder output $\mathbf{c}_\tp$ and Decoder latent state $\h_\tp$
- are input to another Neural Network
- which produces output $\hat\y_\tp$




The "Choose" box implements an *Attention* mechanism, which allows the Decoder
- to **attend to** the part of Input $\x$ (represented via some Encoder latent state $\bar\h_{(\bar\tt)}$)
- that is *relevant* for producing $\hat \y_\tp$
- exactly when it is needed

This seems very natural to a human
- rather than memorizing details (e.g., the big dictionary $\bar\h_{(\bar T)}$ in the architecture without Attention)
- we refer back to the context
- focusing of only the part that is immediately needed

The discussion of the **implementation** of Attention will be deferred to a
later module [Attention lookup](Attention_Lookup.ipynb).

For now, think of the "Choose" box as a Context Sensitive Memory (as described in the module on [Neural Programming](Neural_Programming.ipynb#Soft-Lookup))
- Like a Python `dict`
    - Collection of key/value pairs: $\langle \bar\h_{(\bar \tt)}, \bar\h_{(\bar \tt)} \rangle$
    - Key is equal to value; they are latent states of the Encoder
- But with *soft* lookup
    - The current Decoder state $\h_\tp$ is presented to the CSM 
        - Called the *query*
        - Is matched across each key of the dict (i.e., a latent state $\bar \h_{(\bar \tt)}$)
    - The CSM returns an approximate match of the query to a *key* of the `dict`
        - The distance between the query and each key in the CSM is computed
        - The Soft Lookup returns a *weighted* (by inverse distance) sum of the *values* in the CSM `dict`

# Multi-head attention: two heads are better than one

Perhaps when generating the output for position $\tt$ of the output sequence
- we need to attend to *more than one* position of the sequence being attended to
    - need to know both gender and plurality of subject
- that is: we want an Attention layer to output multiple items.

We can attend to $n$ positions
- by creating $n$ separate Attention mechanisms
- each one called a *head*

This behavior is referred to as *Multi-head attention*

This type of behavior is common to many layer types in a Neural Network
- a Dense layer $l$ may produce a vector $\y_\llp$ where $n_\llp \gt 1$
- a Convolutional layer $l$ may produce outputs (for each spatial location) for many channels

We have referred to this as layer $\ll$ producing $n_\llp$ *features*.

It would be natural for an Attention layer to output many "features" to enable attention to many positions.

In practice, this is sometimes (always ?) not done
- Model architectures (e.g., the Transformer) are simplified when the inputs/outputs of each sub-component
- have the same length
- often denoted as $d$ or $d_\text{model}$ in the Transformer


When a Transformer needs to attend to $n$ positions
- it uses $n$ Attention heads
- each outputting a vector of length $\frac{d}{n}$
- which are concatenated together to produce a single output of length $d$

In essence
- we create $n$ "mini-attention" heads
    - we uses queries
    - and input sequences
    -which are a *fraction* of the original lengths
- the outputs of the $n$ mini-heads can be combined into an output of length $d$

So multi-head attention is compatible (in terms of shape of input and output) with
single head attention.

When we have $n$ heads
- Rather than having one Attention head operating on vectors of length $d$
    - producing an output of length $d$ (weighted sum of values in the CSM)
- We create $n$ Attention heads operating on vectors (keys, values, queries) of length $d \over n$.
    - Output of these smaller heads are values, and hence also of length $d \over n$
- The final output concatenates these $n$ outputs into a single output of length $d$
    - identical in length to the single head
- we project each of these length $d$ vector into vectors of length $d \over n$


The picture shows $n$ Attention heads.

Note that each head is working on vectors of length $\frac{d}{n}$ rather than
original dimensions $d$.
- variables with superscript $(j)$ are of fractional length

Details are deferred to the module [Attention lookup](Attention_Lookup.ipynb).


Each head $j$ uniquely transforms the query $\h_\tp$ and the key/value pairs $\bar{\h}_{(1)} \ldots \bar{\h}_{(\bar{T})}$ being queried.
- into $\h^{(j)}_\tp$ and the key/value pairs $\bar{\h}^{(j)}_{(1)} \ldots \bar{\h}^{(j)}_{(\bar{T})}$
- all vectors/vector elements with superscripts are of length $\frac{d}{n}$ rather than the original length $d$

<table>
    <tr>
        <th><center>Decoder Multi-head Attention</center></th>
    </tr>
    <tr>
        <td><img src="images/Multihead_attention.png" width=80%></td>
    </tr>
</table>

# Teacher forcing: another use case for Masked Self Attention

You may wonder how it is even practically possible for a Decoder to refer to the future.

When using *Teacher Forcing* for **training**
- the Decoder does not use the *predicted* target sequence $\hat \y_{(1:T)}$
- the Decoder uses the *actual* target sequence $\y_{(1:T)}$
    - hence, "future" positions $\tt' \ge \tt$ are available
- this prevents a single mis-prediction at position $\tt$ from cascading and ruining all future output
    - facilitates training
- at inference time: the Decoder works on the *predicted* Target sequence.

In the diagram below, we illustrate (lower right) how the Decoder input changes between Training and Test/Inference time.

<table>
    <tr>
        <th><center>Sequence to Sequence: training (teacher forcing) + inference: No attention</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_seq2seq.png"></td>
    </tr>
</table>

# Transformers (preview)

Using various flavor of Attention
- we have replaced the Encoder and Decoder RNN's in an Encoder/Decoder architecture
- with a new layer type
    - direct function approach to sequences
    
This new architecture is the basis for the *Transformer* layer
- a key advance in modern Deep Learning, particularly for NLP
- which we will study in more depth in a subsequent module


# Conclusion

We recognized that the Decoder function responsible for generating Decoder output $\hat{\y}_\tp$
$$
\hat{\y}_\tp = D( \h_\tp; \mathbf{s})
$$

was quite rigid when it ignored argument $\mathbf{s}$.

This rigidity forced Decoder latent state $\h_\tp$ to assume the additional responsibility of including Encoder context.

Attention was presented as a way to obtain Encoder context through argument $\mathbf{s}$.

In [2]:
print("Done")

Done
